In [171]:
'''Current issues for all functions:
    Improve documentation for debit spreads
    Need to get FairPrice <= 0.01 to be = 0.01
    Need to arrange Volume according to ticker and popularity (Mean volume, ig) -> (Done)
    Get float to round upto 2 decimal places
    Make the entire thing wayyy more efficient
'''
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
import numpy as np
import yfinance as yf
s = yf.Ticker("GNUS")
print(s.history(period='today')['Close'])
opt = s.option_chain('2020-08-20')
calls = opt.calls
#print(int(calls['volume'].mean()))
calls['Fair Price'] = (calls['ask']+calls['bid'])/2
puts = opt.puts
puts['Fair Price'] = (puts['ask']+puts['bid'])/2


Date
2020-07-13   2.230
Name: Close, dtype: float64


In [172]:
def Call_Credit_Spread():
    
    c = calls[calls['volume']>=int(calls['volume'].mean())]
    l = c[['strike', 'Fair Price']]
    m = l.set_index('strike').to_dict()
    n = m['Fair Price'] 
    
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio

# best_ratio: Dictionary to maintain the best available trade at a given time
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}

    # First loop to check what to short (sell)   
    for short in n:
        
        # Second loop to check what we're buying (long)
        for long in n:
            
            # In a credit call spread the long Strike cannot be lower or equal to the short strike price
            if long<=short:
                continue
            else:
                
                # Declaring variables to work with
                strikeShort = short
                strikeLong = long
                
                premiumShort = n[short]
                premiumLong = n[long]
                
                # Figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                MaxReward = premiumShort - premiumLong
                
                #print("MaxReward = ", MaxReward)
                
                MaxRisk = (strikeLong - strikeShort - MaxReward)
                
                #print("MaxRisk = ", MaxRisk)
                
                # Cannot divide by 0.
                if MaxReward == 0:
                    continue
                
                # We don't want the reward to be in negative, which means that no matter what, we won't be profitting
                elif MaxReward>0:
                    
                    Ratio =  float('{:,.2f}'.format(MaxRisk/MaxReward))
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        best_ratio['Long(Buy)']=long
                        best_ratio['Short(Sell)']=short
                        best_ratio['Risk/Reward Ratio']=Ratio
                        best_ratio['Short Premium'] = n[short]
                        best_ratio['Long Premium'] = n[long]
                        best_ratio['maxRisk'] = MaxRisk
                        best_ratio['maxReward'] = MaxReward
                        
    # returning the final best trade available as a call credit spread              
    return print("Best Call Credit Spread: ", best_ratio)
        



In [173]:
def Put_Credit_Spread():
    
    p = puts[puts['volume']>=int(puts['volume'].mean())]
    l = p[['strike', 'Fair Price']]
    m = l.set_index('strike').to_dict()
    n = m['Fair Price']
    
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio

    # best_ratio: Dictionary to maintain the best available trade at a given time
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    
    # First loop to check what to long (buy) 
    for long in n:
        
        # Second loop to check what we're selling(short)
        for short in n:
            
            # In a put credit spread the strike price of what we're buying cannot be higher than strike price of what we're selling
            if long >= short:
                continue
                
            else:
                
                #Decalring various variables
                strikeShort = short
                strikeLong = long
                premiumShort = n[short]
                premiumLong = n[long]
                
                #figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                MaxReward = premiumShort - premiumLong      
                #print("MaxReward = ", MaxReward)
                
                MaxRisk = (strikeLong - strikeShort - MaxReward)
                MaxRisk*=-1
                #print("MaxRisk = ", MaxRisk)
                
                # Cannot divide by 0.
                if MaxReward == 0:
                    continue
                    
                # Don't want negative Rewards...  
                elif MaxReward>0:
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        best_ratio['Long(Buy)']=long
                        best_ratio['Short(Sell)']=short
                        best_ratio['Risk/Reward Ratio']=Ratio
                        best_ratio['Short Premium'] = n[short]
                        best_ratio['Long Premium'] = n[long]
                        best_ratio['maxRisk'] = MaxRisk
                        best_ratio['maxReward'] = MaxReward
                        
                        
    return print("Best Put Credit Spread: ", best_ratio)

In [174]:
def Put_Debit_Spread():
    p = puts[puts['volume']>=int(puts['volume'].mean())]
    l = p[['strike', 'Fair Price']]
    m = l.set_index('strike').to_dict()
    n = m['Fair Price']

#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    for long in n:

        for short in n:
            if long<=short:
                continue
            else:
                strikeShort = short
                strikeLong = long
                #Selling the cheapest one and then buying the more expensive one, one by one
                premiumShort = n[short]
                premiumLong = n[long]
                #figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                MaxReward = strikeLong - strikeShort - premiumLong - premiumShort
#                 print("MaxReward = ", MaxReward)
                MaxRisk = premiumLong - premiumShort
#                 MaxRisk*=-1
#                 print("MaxRisk = ", MaxRisk)
                if MaxReward == 0:
                    continue
                elif MaxReward>0:
                    #We check this to ignore the cases where the oremium is higher for a higher strikeprice in the money
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        best_ratio['Long(Buy)']=long
                        best_ratio['Short(Sell)']=short
                        best_ratio['Risk/Reward Ratio']=Ratio
                        best_ratio['Short Premium'] = n[short]
                        best_ratio['Long Premium'] = n[long]
                        best_ratio['maxRisk'] = MaxRisk
                        best_ratio['maxReward'] = MaxReward
                        
                        
                #print("Ratio = ", Ratio)
    return print("Best Put Debit Spread: ", best_ratio)

In [175]:
#Put_Debit_Spread()

In [176]:
def Call_Debit_Spread():
    
    c = calls[calls['volume']>=int(calls['volume'].mean())]
    l = c[['strike', 'Fair Price']]
    m = l.set_index('strike').to_dict()
    n = m['Fair Price'] 

#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    for long in n:
        for short in n:
            if long>=short:
                continue
            else:
                strikeShort = short
                strikeLong = long
                #Selling the cheapest one and then buying the more expensive one, one by one
                premiumShort = n[short]
                premiumLong = n[long]
                #figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                

                MaxRisk = premiumLong - premiumShort
#                 print("MaxRisk = ", MaxRisk)
                MaxReward = strikeLong - strikeShort - MaxRisk
                MaxReward*=-1
#                 print("MaxReward = ", MaxReward)
                if MaxReward == 0:
                    continue
                elif MaxReward>0:
                    #We check this to ignore the cases where the oremium is higher for a higher strikeprice in the money
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        best_ratio['Long(Buy)']=long
                        best_ratio['Short(Sell)']=short
                        best_ratio['Risk/Reward Ratio']=Ratio
                        best_ratio['Short Premium'] = n[short]
                        best_ratio['Long Premium'] = n[long]
                        best_ratio['maxRisk'] = MaxRisk
                        best_ratio['maxReward'] = MaxReward
                        
                        
                #print("Ratio = ", Ratio)
    return print("Best Call Debit Spread: ", best_ratio)

In [177]:
def Best():
    Call_Credit_Spread()
    print("\n")
    Put_Credit_Spread()
    print("\n")
    Call_Debit_Spread()
    print("\n")
    Put_Debit_Spread()

In [178]:
Best()

Best Call Credit Spread:  {'Short(Sell)': 2.0, 'Long(Buy)': 3.0, 'Risk/Reward Ratio': 2.33, 'Short Premium': 0.7250000000000001, 'Long Premium': 0.42500000000000004, 'maxRisk': 0.7, 'maxReward': 0.30000000000000004}


Best Put Credit Spread:  {'Short(Sell)': 3.0, 'Long(Buy)': 2.0, 'Risk/Reward Ratio': 2.2500000000000004, 'Short Premium': 1.4, 'Long Premium': 0.6000000000000001, 'maxRisk': 1.7999999999999998, 'maxReward': 0.7999999999999998}


Best Call Debit Spread:  {'Short(Sell)': 4.0, 'Long(Buy)': 3.0, 'Risk/Reward Ratio': 0.07246376811594206, 'Short Premium': 0.3, 'Long Premium': 0.42500000000000004, 'maxRisk': 0.12500000000000006, 'maxReward': 1.725}


Best Put Debit Spread:  {'Short(Sell)': 0, 'Long(Buy)': 0, 'Risk/Reward Ratio': 2147483647, 'Short Premium': 0, 'Long Premium': 0, 'maxRisk': 0, 'maxReward': 0}
